In [1]:
import numpy as np
import xarray as xr
import gsw

In [2]:
from osprey.actions.reader import reader_nemo
from osprey.means.means import timemean

In [3]:
data = reader_nemo(expname='FE01', startyear=2000, endyear=2000)

2024-09-23 09:15:21,656 - INFO - Files to be loaded ['/ec/res4/scratch/itas/ece4/FE01/output/nemo/FE01_oce_1m_T_2000-2000.nc']


In [4]:
data

<xarray.Dataset> Size: 143MB
Dimensions:      (y: 148, x: 180, z: 31, time: 12)
Coordinates:
    lat          (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
    lon          (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
  * z            (z) float32 124B 5.0 15.0 25.0 ... 4.25e+03 4.75e+03 5.25e+03
  * time         (time) object 96B 2000-01-16 12:00:00 ... 2000-12-16 12:00:00
Dimensions without coordinates: y, x
Data variables: (12/22)
    e3t          (time, z, y, x) float32 40MB dask.array<chunksize=(1, 31, 148, 180), meta=np.ndarray>
    thetao       (time, z, y, x) float32 40MB dask.array<chunksize=(1, 31, 148, 180), meta=np.ndarray>
    so           (time, z, y, x) float32 40MB dask.array<chunksize=(1, 31, 148, 180), meta=np.ndarray>
    tos          (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
    sos          (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
    zos          (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
    ...           ...
    qt_oce       (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
    sfx          (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
    taum         (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
    windsp       (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
    precip       (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
    snowpre      (time, y, x) float32 1MB dask.array<chunksize=(1, 148, 180), meta=np.ndarray>
Attributes:
    name:         output/nemo/FE01_oce_1m_T
    description:  ECE4/NEMO T grid variables
    title:        ECE4/NEMO T grid variables
    Conventions:  CF-1.6
    timeStamp:    2024-Sep-03 12:24:43 GMT
    uuid:         fca3e523-4867-403c-9c3f-2d2796da6f96

In [22]:
meandata = data.mean(dim=['time'])

In [24]:
# Extract temperature and salinity fields
z = data['z']
lat = data['lat']
pressure = gsw.p_from_z(-z, lat)
temperature = data['thetao']
salinity = data['so']
rho = gsw.density.rho(salinity, temperature, pressure)

# Compute density gradient (dz is the vertical difference in depth)
dz = data['z'].diff(dim='z')
grad_rho = rho.diff(dim='z') / dz

# Find unstable zones (density should increase with depth)
unstable_zones = grad_rho.where(grad_rho < 0.0, drop=True)

# If unstable zones exist, homogenize
if not unstable_zones.isnull().all():
    for idx in unstable_zones.z:
        idx_next = idx + 1
        # Homogenize salinity and temperature between unstable layers
        salinity.loc[dict(z=slice(idx, idx_next))] = salinity.sel(z=slice(idx, idx_next)).mean(dim='z')
        temperature.loc[dict(z=slice(idx, idx_next))] = temperature.sel(z=slice(idx, idx_next)).mean(dim='z')
    
data['thetao'] = temperature
data['so'] = salinity

KeyError: 'Indexing with a boolean dask array is not allowed. This will result in a dask array of unknown shape. Such arrays are unsupported by Xarray.Please compute the indexer first using .compute()'